In [1]:
import os
import pprint
import re
import time
import numpy as np
from numpy.lib import recfunctions as rfn
from pathlib import Path
from pxr import Usd, UsdGeom, Vt, Gf
from pybg3 import pak, lsf, _pybg3

In [2]:
def checktime(msg, cb):
    t_start = time.time()
    rv = cb()
    t_end = time.time()
    print(f"{msg}: {t_end - t_start}")
    return rv

In [3]:
BG3_ROOT = Path(os.environ.get("BG3_DATA", os.path.expanduser("~/l/bg3/Data")))
GUSTAV = checktime("Gustav.pak", lambda: pak.PakFile(BG3_ROOT / "Gustav.pak"))
SHARED = checktime("Shared.pak", lambda: pak.PakFile(BG3_ROOT / "Shared.pak"))
ENGINE = checktime("Engine.pak", lambda: pak.PakFile(BG3_ROOT / "Engine.pak"))
MODELS = checktime("Models.pak", lambda: pak.PakFile(BG3_ROOT / "Models.pak"))
VIRTUAL_TEXTURES = checktime("VirtualTextures.pak", lambda: pak.PakFile(BG3_ROOT / "VirtualTextures.pak"))

Gustav.pak: 0.058767080307006836
Shared.pak: 0.0038907527923583984
Engine.pak: 0.00034117698669433594
Models.pak: 0.060588836669921875
VirtualTextures.pak: 0.05281996726989746


In [4]:
data = GUSTAV.file_data("Generated/Public/GustavDev/Assets/HLOD/BGH_SteelWatchFoundry_B/HLOD_1_0_0_1.GR2")
granny = _pybg3._GrannyReader.from_data(data)

In [5]:
root = granny.root
for name in dir(root):
  print(f"{name}: {root.__getattr__(name)}")
print(dir(root.ArtToolInfo))
print(root.ArtToolInfo.ArtToolPointerSize)
print(root.ArtToolInfo.UpVector)

Animations: <pybg3._pybg3._GrannyPtrSpan object at 0x1192a8e30>
ArtToolInfo: <pybg3._pybg3._GrannyPtr object at 0x1192a8e30>
ExporterInfo: None
ExtendedData: None
FromFileName: D:\Jenkins\workspace\Repo\FB\Main_BuildServer_HLOD\FW4\Stable\LSProjects\Apps\Gustav\Data\Generated\Public\GustavDev\Assets\HLOD\BGH_SteelWatchFoundry_B\HLOD_1_0_0_1.fbx
Materials: <pybg3._pybg3._GrannyPtrSpan object at 0x1192a8e30>
Meshes: <pybg3._pybg3._GrannyPtrSpan object at 0x1192a8e30>
Models: <pybg3._pybg3._GrannyPtrSpan object at 0x1192a8e30>
Skeletons: <pybg3._pybg3._GrannyPtrSpan object at 0x1192a8e30>
Textures: <pybg3._pybg3._GrannyPtrSpan object at 0x1192a8e30>
TrackGroups: <pybg3._pybg3._GrannyPtrSpan object at 0x1192a8e30>
TriTopologies: <pybg3._pybg3._GrannyPtrSpan object at 0x1192a8e30>
VertexDatas: <pybg3._pybg3._GrannyPtrSpan object at 0x1192a8e30>
['ArtToolMajorRevision', 'ArtToolMinorRevision', 'ArtToolPointerSize', 'BackVector', 'ExtendedData', 'FromArtToolName', 'Origin', 'RightVector', 'Un

In [6]:
for mesh in root.Meshes:
  print(mesh.Name)
  vd = mesh.PrimaryVertexData
  nda = np.array(vd.Vertices, copy = False)
  new_arr = rfn.structured_to_unstructured(nda[["f0", "f1", "f2"]], copy = False)
  print(vd.VertexComponentNames)
  print(f"{vd.Vertices}: {len(vd.Vertices)} vertices")
  print(f"vertex data: {nda}")
  print(f"vertex data shape: {nda.shape}")
  print(f"vertex data dtype: {nda.dtype}")
  print(f"reshaped vertex data: {new_arr} {new_arr.shape} {new_arr.dtype}")
  print(dir(vd))

GustavDev_BGH_SteelWatchFoundry_B_HLOD_1_0_0_1_6
<pybg3._pybg3._GrannyDirectSpan object at 0x11921de70>: 61136 vertices
vertex data: [(-22.55572  ,  2.00884  , -20.178854 ,  17575,  -765, -15485, 22900, 0.3567, 0.5986)
 (-22.265688 ,  1.6835029, -20.419418 ,   7340,  5968, -13937, 28106, 0.3604, 0.5986)
 (-22.633457 ,  2.1683881, -20.025026 ,   7376,  5392, -14106, 28128, 0.355 , 0.5986)
 ...
 (  7.3702993, -1.881639 ,  -7.500201 ,  -8471,  -142, -31652,   244, 0.8667, 0.4783)
 (  7.585391 , -1.8492799,  -7.5070524, -10062, 16950, -26048,  2576, 0.865 , 0.4783)
 (  7.5191507, -1.8988459,  -7.562937 ,  -9586, 10161, -29620,  1103, 0.8657, 0.4785)]
vertex data shape: (61136,)
vertex data dtype: [('f0', '<f4'), ('f1', '<f4'), ('f2', '<f4'), ('f3', '<i2'), ('f4', '<i2'), ('f5', '<i2'), ('f6', '<i2'), ('f7', '<f2'), ('f8', '<f2')]
reshaped vertex data: [[-22.55572     2.00884   -20.178854 ]
 [-22.265688    1.6835029 -20.419418 ]
 [-22.633457    2.1683881 -20.025026 ]
 ...
 [  7.3702993  -1.

In [7]:
g_mesh = root.Meshes[0]
g_vertices = np.array(g_mesh.PrimaryVertexData.Vertices, copy = False)
g_indices = np.array(g_mesh.PrimaryTopology.Indices16, copy = False)
print(g_mesh.Name)
pprint.pp(dir(g_mesh.PrimaryVertexData.Vertices[0]))

GustavDev_BGH_SteelWatchFoundry_B_HLOD_1_0_0_1_6
['Position', 'QTangent', 'TextureCoordinates0']


In [8]:
u_stage = Usd.Stage.CreateNew("tmp/granny_test.usda")
u_mesh = UsdGeom.Mesh.Define(u_stage, "/mesh")

In [9]:
u_points = u_mesh.CreatePointsAttr()
g_positions = rfn.structured_to_unstructured(g_vertices[["f0", "f1", "f2"]], copy = False)
vt_vertices = Vt.Vec3fArray.FromNumpy(g_positions)
vt_indices = Vt.IntArray.FromNumpy(g_indices)
vt_face_counts = Vt.IntArray.FromNumpy(np.full(g_positions.shape[0], 3, dtype = np.int32))
u_points.Set(vt_vertices)
u_vertex_face_counts = u_mesh.CreateFaceVertexCountsAttr()
u_vertex_face_counts.Set(vt_face_counts)
u_vertex_indices = u_mesh.CreateFaceVertexIndicesAttr()
u_vertex_indices.Set(vt_indices)

True

In [10]:
u_stage.GetRootLayer().Save()

True

In [11]:
for mesh in root.Meshes:
  print(mesh.Name)

GustavDev_BGH_SteelWatchFoundry_B_HLOD_1_0_0_1_6
GustavDev_BGH_SteelWatchFoundry_B_HLOD_1_0_0_1_5
GustavDev_BGH_SteelWatchFoundry_B_HLOD_1_0_0_1_4
GustavDev_BGH_SteelWatchFoundry_B_HLOD_1_0_0_1_3
GustavDev_BGH_SteelWatchFoundry_B_HLOD_1_0_0_1_2
GustavDev_BGH_SteelWatchFoundry_B_HLOD_1_0_0_1_1
GustavDev_BGH_SteelWatchFoundry_B_HLOD_1_0_0_1_0
GustavDev_BGH_SteelWatchFoundry_B_HLOD_1_0_0_1_emissive_0
GustavDev_BGH_SteelWatchFoundry_B_HLOD_1_0_0_1_two_sided_0


In [12]:
squirrel = MODELS.file_data("Generated/Public/Shared/Assets/Nature/Plants/NAT_Coastal_Plant_Bush_Pine_ABC/Resources/NAT_Coastal_Plant_Bush_Pine_Small_A.GR2")
print(len(squirrel))
g_squirrel = _pybg3._GrannyReader.from_data(squirrel)
print(dir(g_squirrel.root))
for mesh in g_squirrel.root.Meshes:
  print(mesh.Name)
for model in g_squirrel.root.Models:
  print(model.Name)

139204
['Animations', 'ArtToolInfo', 'ExporterInfo', 'ExtendedData', 'FromFileName', 'Materials', 'Meshes', 'Models', 'Skeletons', 'Textures', 'TrackGroups', 'TriTopologies', 'VertexDatas']
NAT_Coastal_Plant_Bush_Pine_Small_A_Mesh_LOD3
NAT_Coastal_Plant_Bush_Pine_Small_A_Mesh_LOD2
NAT_Coastal_Plant_Bush_Pine_Small_A_Mesh_LOD1
NAT_Coastal_Plant_Bush_Pine_Small_A_Mesh
Dummy_NAT_Coastal_Plant_Bush_Pine_Small_A


In [13]:
Gf.Quatf(0,0,0,1)

Gf.Quatf(0.0, Gf.Vec3f(0.0, 0.0, 1.0))

In [14]:
patch_data = GUSTAV.file_data("Mods/Gustav/Levels/TUT_Avernus_C/Terrains/2b55a30d-aca1-447f-8f4e-4d9215958607_2_0.patch")
patch = _pybg3._PatchFile.from_data(patch_data)

In [15]:
height = np.array(patch.heightfield, copy = False)
print(height.shape)
print(np.array(patch.heightfield, copy = False))

(65, 18)
[[0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 ...
 [7.6419263 5.7370896 4.1396384 ... 0.        0.        0.       ]
 [7.9959707 6.4323583 5.076839  ... 0.        0.        0.       ]
 [7.6588874 6.927524  5.5049357 ... 0.        0.        0.       ]]


In [16]:
points = np.dstack((
  np.repeat(
    np.arange(0, height.shape[0]).reshape(-1, 1),
    height.shape[1],
    axis = 1
  ),
  height,
  np.repeat(
    np.arange(0, height.shape[1]).reshape(1, -1),
    height.shape[0],
    axis = 0
  ),
)).reshape(-1, 3)

In [17]:
points

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  2.],
       ...,
       [64.,  0., 15.],
       [64.,  0., 16.],
       [64.,  0., 17.]])

In [18]:
# copilot lawl
indices = np.arange(patch.local_rows * patch.local_cols).reshape(patch.local_rows, patch.local_cols)
triangles = np.array([indices[:-1, :-1], indices[:-1, 1:], indices[1:, :-1], indices[1:, :-1], indices[:-1, 1:], indices[1:, 1:]])
index_buffer = triangles.transpose(1, 2, 0).reshape(-1)

print(index_buffer)

[   0    1   18 ... 1168 1151 1169]


In [19]:
GUSTAV._lspk.num_parts()

1

In [20]:
def vt_test():
  for name in VIRTUAL_TEXTURES.files():
    if VIRTUAL_TEXTURES.file_part(name) > 0:
      print(name)
      data = VIRTUAL_TEXTURES.file_data(name)
      os.makedirs(f"tmp/{os.path.dirname(name)}", exist_ok = True)
      with open(f"tmp/{name}", "wb") as f:
        f.write(data)
      break
vt_test()

Generated/Public/VirtualTextures/Albedo_Normal_Physical_3_33b4a4abc3fa10dbd2866bb430463254.gtp


FileNotFoundError: [Errno 2] No such file or directory: 'tmp/Generated/Public/VirtualTextures/Albedo_Normal_Physical_3_33b4a4abc3fa10dbd2866bb430463254.gtp'